In [1]:
from vantage6.client import Client
from verticox.client import VerticoxClient
import os

/home/dsmits/.cache/pypoetry/virtualenvs/verticox-mILc84M0-py3.10/lib/python3.10/site-packages/vantage6/common/encryption.py:25: CryptographyDeprecationWarning: Use PrivateKeyTypes instead
  from cryptography.hazmat.primitives.asymmetric.types import PRIVATE_KEY_TYPES


In [2]:
user = os.environ["V6_USER"]
password = os.environ["V6_PASSWORD"]

In [3]:
client = Client("https://v6server.carrier-mu.src.surf-hosted.nl", 443)

 Welcome to
                  _                     __  
                 | |                   / /  
__   ____ _ _ __ | |_ __ _  __ _  ___ / /_  
\ \ / / _` | '_ \| __/ _` |/ _` |/ _ \ '_ \ 
 \ V / (_| | | | | || (_| | (_| |  __/ (_) |
  \_/ \__,_|_| |_|\__\__,_|\__, |\___|\___/ 
                            __/ |           
                           |___/            

 --> Join us on Discord! https://discord.gg/rwRvwyK
 --> Docs: https://docs.vantage6.ai
 --> Blog: https://vantage6.ai
------------------------------------------------------------
Cite us!
If you publish your findings obtained using vantage6, 
please cite the proper sources as mentioned in:
https://vantage6.ai/vantage6/references
------------------------------------------------------------


In [4]:
client.authenticate(user, password)

Authenticating user djura...
Starting new HTTPS connection (1): v6server.carrier-mu.src.surf-hosted.nl:443
https://v6server.carrier-mu.src.surf-hosted.nl:443 "POST /api/token/user HTTP/1.1" 200 770
Successfully authenticated
Making request: GET | https://v6server.carrier-mu.src.surf-hosted.nl:443/api/user/9 | None
Starting new HTTPS connection (1): v6server.carrier-mu.src.surf-hosted.nl:443
https://v6server.carrier-mu.src.surf-hosted.nl:443 "GET /api/user/9 HTTP/1.1" 200 4386
Making request: GET | https://v6server.carrier-mu.src.surf-hosted.nl:443/api/organization/2 | None
Starting new HTTPS connection (1): v6server.carrier-mu.src.surf-hosted.nl:443
https://v6server.carrier-mu.src.surf-hosted.nl:443 "GET /api/organization/2 HTTP/1.1" 200 1488
 --> Succesfully authenticated
 --> Name: Djura (id=9)
 --> Organization: djura (id=2)


In [5]:
client.setup_encryption("private_key.pem")

Loading private key
Making request: GET | https://v6server.carrier-mu.src.surf-hosted.nl:443/api/organization/2 | None
Starting new HTTPS connection (1): v6server.carrier-mu.src.surf-hosted.nl:443
https://v6server.carrier-mu.src.surf-hosted.nl:443 "GET /api/organization/2 HTTP/1.1" 200 1488
Public key matches the server key! Good to go!


In [7]:
collaborations = client.collaboration.list()

for c in collaborations:
    print(f"Name: {c['name']} id: {c['id']}")

pagination for scope `organization` not available
Making request: GET | https://v6server.carrier-mu.src.surf-hosted.nl:443/api/organization/2/collaboration | None
Starting new HTTPS connection (1): v6server.carrier-mu.src.surf-hosted.nl:443
https://v6server.carrier-mu.src.surf-hosted.nl:443 "GET /api/organization/2/collaboration HTTP/1.1" 200 55881
Name: test-infra id: 1


In [8]:
verticox_client = VerticoxClient(client)

pagination for scope `organization` not available
Making request: GET | https://v6server.carrier-mu.src.surf-hosted.nl:443/api/organization/2/collaboration | None
Starting new HTTPS connection (1): v6server.carrier-mu.src.surf-hosted.nl:443
https://v6server.carrier-mu.src.surf-hosted.nl:443 "GET /api/organization/2/collaboration HTTP/1.1" 200 55881


In [9]:
task = verticox_client.get_column_names()

Making request: GET | https://v6server.carrier-mu.src.surf-hosted.nl:443/api/node | {'page': 1, 'per_page': 20, 'include': ['metadata'], 'name': None, 'organization_id': None, 'collaboration_id': None, 'ip': None, 'last_seen_from': None, 'last_seen_till': None, 'status': 'online'}
Starting new HTTPS connection (1): v6server.carrier-mu.src.surf-hosted.nl:443
https://v6server.carrier-mu.src.surf-hosted.nl:443 "GET /api/node?page=1&per_page=20&include=metadata&status=online HTTP/1.1" 200 1525

                    task = self.v6client.task.create(collaboration=1,
                                             organizations=[4, 5, 6],
                                             name=,
                                             image=harbor.carrier-mu.src.surf-hosted.nl/carrier/verticox,
                                             description=,
                                             input={'method': 'column_names', 'master': False, 'kwargs': {}},
                                     

In [12]:
results = task.get_result()
results

--> Attempting to decrypt results!
Making request: GET | https://v6server.carrier-mu.src.surf-hosted.nl:443/api/result/1208 | {}
Starting new HTTPS connection (1): v6server.carrier-mu.src.surf-hosted.nl:443
https://v6server.carrier-mu.src.surf-hosted.nl:443 "GET /api/result/1208 HTTP/1.1" 200 2699
Decrypting input
Encryption/decryption failed.
Decrypting result
Decrypted shared key: b'P\x07\xd0A3wum3\\\xcb\xbe\xcfTe.ne\x93\xc1B\xdb$\xf8\x8f \xae\xba\xe1\xd5\xa8\x0f'
No data format specified. Assuming input data is pickle format
Received a result
--> Attempting to decrypt results!
Making request: GET | https://v6server.carrier-mu.src.surf-hosted.nl:443/api/result/1209 | {}
Starting new HTTPS connection (1): v6server.carrier-mu.src.surf-hosted.nl:443
https://v6server.carrier-mu.src.surf-hosted.nl:443 "GET /api/result/1209 HTTP/1.1" 200 2663
Decrypting input
Encryption/decryption failed.
Decrypting result
Decrypted shared key: b')\xe8:`\xcc%\xfa"\r\xf0W\x15\xd3\x04\xea\x16Rt\xdbKo|\x82,\x

[Result(organization=4, content=['event_happened', 'event_time'], log="/root/.cache/pypoetry/virtualenvs/verticox-9TtSrW0h-py3.10/lib/python3.10/site-packages/vantage6/common/encryption.py:25: CryptographyDeprecationWarning: Use PrivateKeyTypes instead\n  from cryptography.hazmat.primitives.asymmetric.types import PRIVATE_KEY_TYPES\ninfo > wrapper for verticox\ninfo > Reading input file /mnt/data/task-000001208/input\ninfo > Reading token file '/mnt/data/task-000001208/token'\ninfo > Using '/mnt/data/default.parquet' as database\ninfo > Dispatching ...\ninfo > Module 'verticox' imported!\ninfo > Running a regular container\ninfo > Writing output to /mnt/data/task-000001208/output\n"),
 Result(organization=5, content=['age'], log="/root/.cache/pypoetry/virtualenvs/verticox-9TtSrW0h-py3.10/lib/python3.10/site-packages/vantage6/common/encryption.py:25: CryptographyDeprecationWarning: Use PrivateKeyTypes instead\n  from cryptography.hazmat.primitives.asymmetric.types import PRIVATE_KEY_TYP

In [13]:
for r in results:
    print(f"Organization: {r.organization}, Content: {r.content}")

Organization: 4, Content: ['event_happened', 'event_time']
Organization: 5, Content: ['age']
Organization: 6, Content: ['bmi']


In [18]:
task = verticox_client.fit(feature_columns=['age', 'bmi'],
                    outcome_time_column='event_time',
                    right_censor_column='event_happened',
                    feature_nodes=[5,6],
                    outcome_node=4
                   )


                    task = self.v6client.task.create(collaboration=1,
                                             organizations=[4],
                                             name=,
                                             image=harbor.carrier-mu.src.surf-hosted.nl/carrier/verticox,
                                             description=,
                                             input={'method': 'fit', 'master': True, 'kwargs': {'feature_columns': ['age', 'bmi'], 'event_times_column': 'event_time', 'event_happened_column': 'event_happened', 'datanode_ids': [5, 6], 'central_node_id': 4, 'precision': 1e-05}},
                                             data_format=json,
                                             database=default
                                             )
            
Making request: GET | https://v6server.carrier-mu.src.surf-hosted.nl:443/api/organization/4 | None
Starting new HTTPS connection (1): v6server.carrier-mu.src.surf-hosted.nl:443
https://

In [19]:
results = task.get_result()
results

--> Attempting to decrypt results!
Making request: GET | https://v6server.carrier-mu.src.surf-hosted.nl:443/api/result/1285 | {}
Starting new HTTPS connection (1): v6server.carrier-mu.src.surf-hosted.nl:443
https://v6server.carrier-mu.src.surf-hosted.nl:443 "GET /api/result/1285 HTTP/1.1" 200 1502
Decrypting input
Encryption/decryption failed.
--> Attempting to decrypt results!
Making request: GET | https://v6server.carrier-mu.src.surf-hosted.nl:443/api/result/1285 | {}
Starting new HTTPS connection (1): v6server.carrier-mu.src.surf-hosted.nl:443
https://v6server.carrier-mu.src.surf-hosted.nl:443 "GET /api/result/1285 HTTP/1.1" 200 1589
Decrypting input
Encryption/decryption failed.
--> Attempting to decrypt results!
Making request: GET | https://v6server.carrier-mu.src.surf-hosted.nl:443/api/result/1285 | {}
Starting new HTTPS connection (1): v6server.carrier-mu.src.surf-hosted.nl:443
https://v6server.carrier-mu.src.surf-hosted.nl:443 "GET /api/result/1285 HTTP/1.1" 200 1589
Decryptin

[Result(organization=4, content=b'', log="/root/.cache/pypoetry/virtualenvs/verticox-9TtSrW0h-py3.10/lib/python3.10/site-packages/vantage6/common/encryption.py:25: CryptographyDeprecationWarning: Use PrivateKeyTypes instead\n  from cryptography.hazmat.primitives.asymmetric.types import PRIVATE_KEY_TYPES\ninfo > wrapper for verticox\ninfo > Reading input file /mnt/data/task-000001285/input\ninfo > Reading token file '/mnt/data/task-000001285/token'\ninfo > Using '/mnt/data/default.parquet' as database\ninfo > Dispatching ...\ninfo > Module 'verticox' imported!\ninfo > Running a master-container\ninfo > Working with collaboration_id <1>\ninfo > Start running verticox on features: ['age', 'bmi']\ninfo > My database: default\n[info ] - Starting java containers\n")]

In [20]:
for r in results:
    print(r)

Result(organization=4, content=b'', log="/root/.cache/pypoetry/virtualenvs/verticox-9TtSrW0h-py3.10/lib/python3.10/site-packages/vantage6/common/encryption.py:25: CryptographyDeprecationWarning: Use PrivateKeyTypes instead\n  from cryptography.hazmat.primitives.asymmetric.types import PRIVATE_KEY_TYPES\ninfo > wrapper for verticox\ninfo > Reading input file /mnt/data/task-000001285/input\ninfo > Reading token file '/mnt/data/task-000001285/token'\ninfo > Using '/mnt/data/default.parquet' as database\ninfo > Dispatching ...\ninfo > Module 'verticox' imported!\ninfo > Running a master-container\ninfo > Working with collaboration_id <1>\ninfo > Start running verticox on features: ['age', 'bmi']\ninfo > My database: default\n[info ] - Starting java containers\n")


In [32]:
def get_logs():
    tasks = client.task.list()
    tasks = tasks['data']
    
    for t in tasks:
        results = t['results']
        for r_stub in results:
            result = client.result.get(r_stub['id'])
            
            yield result['log']
            

logs = get_logs()

In [37]:
log = next(logs)
print(log)

--> Attempting to decrypt results!
Making request: GET | https://v6server.carrier-mu.src.surf-hosted.nl:443/api/result/1304 | {}
Starting new HTTPS connection (1): v6server.carrier-mu.src.surf-hosted.nl:443
https://v6server.carrier-mu.src.surf-hosted.nl:443 "GET /api/result/1304 HTTP/1.1" 200 18848
Decrypting input
Encryption/decryption failed.
Decrypting result
Could not decrypt/decode input or result.
Encryption/decryption failed.
--> Failed to deserialize
'str' object has no attribute 'decode'
/root/.cache/pypoetry/virtualenvs/verticox-9TtSrW0h-py3.10/lib/python3.10/site-packages/vantage6/common/encryption.py:25: CryptographyDeprecationWarning: Use PrivateKeyTypes instead
  from cryptography.hazmat.primitives.asymmetric.types import PRIVATE_KEY_TYPES
info > wrapper for verticox
info > Reading input file /mnt/data/task-000001304/input
info > No data format specified. Assuming input data is pickle format
info > Reading token file '/mnt/data/task-000001304/token'
info > Using '/mnt/dat